In [205]:
import numpy as np
import pandas as pd
import datetime as dt

# Importing fundamental data from CSV file
df = pd.read_csv("MSCI Q New.csv")
df = df.drop(["fqtr", "fyearq"], axis = 1)

,gvkey,datadate,atq,ltq,piq,saleq,txtq,conm
0,8544,20000630,301477.00,NaN,NaN,NaN,NaN,PLDT INC
1,8544,20001231,328496.00,241259.00,NaN,NaN,NaN,PLDT INC
2,8544,20010630,NaN,NaN,NaN,NaN,NaN,PLDT INC
3,8544,20011231,307622.00,218041.00,NaN,NaN,NaN,PLDT INC
4,8544,20020630,304901.80,213157.70,NaN,NaN,NaN,PLDT INC
...,...,...,...,...,...,...,...,...
56776,349758,20201231,69883.21,6621.00,-3513.00,6094.0,13.0,ZOMATO LTD
56777,349758,20210331,87035.43,6105.35,-1342.29,6924.4,0.0,ZOMATO LTD
56778,349758,20210630,NaN,NaN,-3590.00,8444.0,17.0,ZOMATO LTD
56779,349758,20210930,173147.00,8358.00,-4349.00,10242.0,NaN,ZOMATO LTD


In [206]:
# Keeping only fundamental data from 2005 onwards
df = df[df.datadate >= 20050101]

,gvkey,datadate,atq,ltq,piq,saleq,txtq,conm
12,8544,20050331,261403.00,207020.00,12714.00,30144.0,3475.0,PLDT INC
13,8544,20050630,263189.00,203885.00,9585.00,30887.0,2151.0,PLDT INC
14,8544,20050930,257074.00,188768.00,10625.00,31424.0,2417.0,PLDT INC
15,8544,20051231,251984.00,177615.00,5691.00,33268.0,-3907.0,PLDT INC
16,8544,20060331,251110.00,174358.00,11277.00,30879.0,2512.0,PLDT INC
...,...,...,...,...,...,...,...,...
56776,349758,20201231,69883.21,6621.00,-3513.00,6094.0,13.0,ZOMATO LTD
56777,349758,20210331,87035.43,6105.35,-1342.29,6924.4,0.0,ZOMATO LTD
56778,349758,20210630,NaN,NaN,-3590.00,8444.0,17.0,ZOMATO LTD
56779,349758,20210930,173147.00,8358.00,-4349.00,10242.0,NaN,ZOMATO LTD


In [207]:
# Importing forex data and setting index to dates
fx = pd.read_csv("exchange_rates.csv")
fx= fx.set_index("ANNDATS")

cur_codes = fx.CURR.unique()

# Generating currency dictionary with the different currencies being the dictionaries keys
curr_dict = {}
for cur in cur_codes:
    info = fx[fx["CURR"] == cur]
    info = info.drop(["CURR"], axis=1)
    curr_dict[cur] = info

curr_dict

{'AED':            EXRAT               CURNAM
 ANNDATS                              
 20000103  3.6729  United Arab Dirhams
 20000104  3.6731  United Arab Dirhams
 20000105  3.6730  United Arab Dirhams
 20000106  3.6729  United Arab Dirhams
 20000107  3.6732  United Arab Dirhams
 ...          ...                  ...
 20211227  3.6726  United Arab Dirhams
 20211228  3.6726  United Arab Dirhams
 20211229  3.6726  United Arab Dirhams
 20211230  3.6726  United Arab Dirhams
 20211231  3.6726  United Arab Dirhams
 
 [5717 rows x 2 columns],
 'ARS':                EXRAT          CURNAM
 ANNDATS                             
 20000103    0.999800  Argentine Peso
 20000104    0.999900  Argentine Peso
 20000105    0.999900  Argentine Peso
 20000106    0.999900  Argentine Peso
 20000107    0.999500  Argentine Peso
 ...              ...             ...
 20211227  102.540001  Argentine Peso
 20211228  102.599998  Argentine Peso
 20211229  102.629997  Argentine Peso
 20211230  102.680000  Argentine 

In [209]:
# Setting index of fundamental dataframe to the date 
df = df.set_index("datadate")

In [210]:
# Generating the D/E, ROA, and ROE ratios for each quarter
df["D/E"] = df["ltq"]/(df["atq"] - df["ltq"])
df["ROA"] = (df["piq"] - df["txtq"])/df["atq"]
df["ROE"] = (df["piq"] - df["txtq"])/(df["atq"] - df["ltq"])

In [211]:
# Retrieving the unique company names
stock_names = df.conm.unique()

# Dictionary for the fundamental data with company names being the dictionary's keys
test_dict = {}
dup = []
for name in stock_names:
    # Getting company subset information from original df
    info = df[df["conm"] == name]
    info = info.drop(["conm"], axis=1)
    # For each company, create an empty column to calculate the SGI
    info["SGI"] = None
    test_dict[name] = info

In [212]:
# For each company, drop duplicates from fundamental data - keep the first entry for two cases (full data), 
# keep last for the other companies 
for name in stock_names:
    test_dict[name] = test_dict[name].reset_index()
    if name in ['BEIJING ENTERPRISES WATER GR','NATL INDUSTRIALIZATION CO']:
        test_dict[name] = test_dict[name].drop_duplicates(subset = ['datadate'], keep = "first")
    else:
        test_dict[name] = test_dict[name].drop_duplicates(subset = ['datadate'], keep = "last")
    # Reset index back to dates
    test_dict[name] = test_dict[name].set_index('datadate')


    info = test_dict[name]
    # Sort dates by date (from earliest to latest)
    dates = info.index.unique().sort_values(ascending = True)
    for j in range(len(dates)):
        # Check if it is the first date available, set growth index to 1, otherwise, perform SGI calculation
        if (j == 0):
            info.loc[dates[j], "SGI"] = 1
        else:
            info.loc[dates[j], "SGI"] = info.loc[dates[j],"saleq"]/info.loc[dates[j-1], "saleq"]
    test_dict[name] = info

/var/folders/l7/9x99jj25655fgy_97606f5740000gn/T/ipykernel_4574/1193468550.py:16: RuntimeWarning: invalid value encountered in double_scalars
  info.loc[dates[j], "SGI"] = info.loc[dates[j],"saleq"]/info.loc[dates[j-1], "saleq"]
/var/folders/l7/9x99jj25655fgy_97606f5740000gn/T/ipykernel_4574/1193468550.py:16: RuntimeWarning: divide by zero encountered in double_scalars
  info.loc[dates[j], "SGI"] = info.loc[dates[j],"saleq"]/info.loc[dates[j-1], "saleq"]


In [213]:
# Import daily price data from CSV file - keep only the dates begining with New Year's Day 2005
daily = pd.read_csv("MSCI Cur Codes.csv")
daily = daily[daily.datadate >= 20050101]

In [215]:
# Set daily price dataframe index to dates
daily = daily.set_index("datadate")
daily = daily.drop("iid", axis=1)

976

In [216]:
# Creating a dictionary for each asset's daily price history
prices_dict = {}
for name in stock_names:
    info = daily[daily["conm"] == name]
    #info = info.drop(["conm", "ajexdi", "trfd"], axis=1)
    
    #Drop company name from the subset 
    info = info.drop(["conm"], axis=1)
    # Drop any rows that have at least 4 columns that are NaN or None 
    info = info.dropna(thresh = 4)
    # Create empty column for the closing price expressed in USD
    info["PriceUSD"] = None
    prices_dict[name] = info

prices_dict

{'PLDT INC':           gvkey curcdd  ajexdi    prccd      trfd PriceUSD
 datadate                                                  
 20050103   8544    PHP     1.0  1390.00  1.078887     None
 20050104   8544    PHP     1.0  1390.00  1.078887     None
 20050105   8544    PHP     1.0  1390.00  1.078887     None
 20050106   8544    PHP     1.0  1355.00  1.078887     None
 20050107   8544    PHP     1.0  1345.00  1.078887     None
 ...         ...    ...     ...      ...       ...      ...
 20110915   8544    PHP     1.0    12.00  2.650862     None
 20110920   8544    PHP     1.0    11.50  2.650862     None
 20110922   8544    PHP     1.0    11.50  2.650862     None
 20110926   8544    PHP     1.0    11.50  2.650862     None
 20111004   8544    PHP     1.0    11.12  2.889249     None
 
 [10657 rows x 6 columns],
 'ANGLO AMERICAN PLATINUM LTD':           gvkey curcdd  ajexdi     prccd      trfd PriceUSD
 datadate                                                   
 20050103  16610    ZAR  1

In [ ]:
# In order to deal with stocks that are potentially dual listed, 
# this function creates new dictionary that keeps the stocks that have at most 1 currency change
def get_stable_stocks(prices_dict, stock_names):
    # Create empty dictionary
    stable_stocks_dict = {}
    for name in stock_names:
        # Counter for the number of times the currecncy is different between subsequent rows 
        counter = 0
        dates = prices_dict[name].index.unique().sort_values(ascending = True)
        for i in range(1, len(dates)):
            # Check whether the currency from the current day to the prior day's currency is different 
            if (prices_dict[name].loc[dates[i], "curcdd"] != prices_dict[name].loc[dates[i-1], "curcdd"]):
                counter += 1
        # If counter is greater than 1, then do not at it to the new dictionary, and continue the for-loop
        if counter > 1:
            continue
        # Otherwise, add existing price information to the new dictionary    
        stable_stocks_dict[name] = prices_dict[name]
    # Once loop has completed, return the stable_stock dictionary   
    return stable_stocks_dict

In [225]:
# Function that converts the daily asset prices into USD 
def price_to_usd(prices_dict, stock_names):
    # Create a new dictionary 
    prices_dict_usd = {}
    # Run for loop across the asset names
    for name in stock_names:
        # Get available dates for asset's history in chronological order
        dates = prices_dict[name].index.unique().sort_values(ascending = True)
        # Create an empty df with the index being the available dates and a single column
        df = pd.DataFrame(index = dates, columns = ["PriceUSD"])
        check = []

        for j in range(len(dates)):
        # For each date available, 
        # try to search the forex dictionary for the exchange rate between the currency at the time and USD            
            try: 
                # Searching the currency dictionary for the exchange rate
                rate = curr_dict[prices_dict[name].loc[dates[j], "curcdd"]].loc[dates[j]][0]
                # Calculate the stock price in USD 
                df.loc[dates[j], "PriceUSD"] = ((prices_dict[name].loc[dates[j],"prccd"]/rate))
                # If rate is available, update the check array with the latest exchange rate 
                check.append(rate)
            except:
                # If the search draws an exception, first check if it is the first date available 
                if (j == 0):
                    # If first available date, break the loop and provide an empty df under that asset name
                    break
                else:
            # If after the first date available, use the last known exchange rate (check array) to determine USD price
                    df.loc[dates[j], "PriceUSD"] = ((prices_dict[name].loc[dates[j],"prccd"]/check[-1]))
                    #continue
                    
        # Assign the dictionary key the update dataframe
        prices_dict_usd[name] = df
    
    # Once loop has completed, return the USD dictionary
    return prices_dict_usd

In [230]:
# Function to drop duplicates by date from dictionary
def drop_dups(prices_dict, stock_names):  
    for name in stock_names:
        prices_dict[name] = prices_dict[name].reset_index()
        prices_dict[name] = prices_dict[name].drop_duplicates(subset = ['datadate'], keep = "first")
        prices_dict[name] = prices_dict[name].set_index('datadate')
    return prices_dict

# Function that computes gross daily returns from given dictionary
def daily_returns(prices_dict, stock_names):   
    for name in stock_names:
        # For each key, add empty column for returns
        prices_dict[name]["Ret"] = None
        dates = prices_dict[name].index.unique().sort_values(ascending = True)
        for j in range(len(dates)):
            # If first available date, set return for that date to zero
            if (j == 0):
                prices_dict[name].loc[dates[j], "Ret"] = 0
            # Otherwise calculate daily returns 
            else:
                prices_dict[name].loc[dates[j], "Ret"] = ((prices_dict[name].loc[dates[j],"price/adj*trfd"]/prices_dict[name].loc[dates[j-1], "price/adj*trfd"])-1)
    # Return update dictionary            
    return prices_dict


In [231]:
def monthly_returns(df, stock_names):
    # Getting a list of the months from Jan 05 to Dec 21 in the form YYYYMM
    months = pd.date_range('2005-01-01','2021-12-31', freq='MS').strftime("%Y%m").tolist()
    # Creating returns dictionary
    returns = {}
    for name in stock_names:
        # Creating empty dataframe 
        bb = pd.DataFrame()
        for month in months: 
            # Retrieve the dates for that dictionary key and convert to a list 
            dates = df[name].index.tolist()
            # Converting available dates to string
            str_dates = [str(k) for k in dates]
            # Boolean array to check for the rows that are in that month 
            bbc = [str(month) in k for k in str_dates]
            # Try searching for all the days that are within that month 
            try:
                monthly = df[name][bbc]
                # Create array with intial value being the month in question
                check = [month]
                # Append the check array with the return for that month. 
                # This is achieved using cumulative products and taking the past element of the cumprod method less one
                check.append((monthly["Ret"].add(1).cumprod(axis=0).iloc[-1])-1)
                # Create pandas time series for this data 
                add = pd.Series(check)
                # Add this monthly information to bb
                bb = bb.append(add, ignore_index = True)
            except:
                # If you cannot find the month, continue the for loop
                continue
        # Monthly returns for the given asset is then bb       
        returns[name] = bb
        # Columns names for the monthly return data are set
        returns[name].columns = ["Month", "MonthlyReturn"]
    # Once looping is complete, return the returns dictionary 
    return returns


# Function that converts monthly data to quarterly data 
def quarterly_returns(returns, stock_names):
    # Create quarterly returns dictionary
    q_returns = {}
    # Create an array for the years 2005-2021
    years = [year for year in range(2005,2022)]
    
    for name in stock_names:
        # Create empy dataframe 
        b = pd.DataFrame()
        # For each year, create months that fall into Q1, Q2, Q3, and Q4
        for year in years:
            q1 = [str(year) + "01", str(year) + "02", str(year) + "03"]
            q2 = [str(year) + "04", str(year) + "05", str(year) + "06"]
            q3 = [str(year) + "07", str(year) + "08", str(year) + "09"]
            q4 = [str(year) + "10", str(year) + "11", str(year) + "12"]
            # Array of each quarter for a given year 
            qs = [q1,q2,q3,q4]

            for q in qs:
                # For each quarter, initialise array with last month of each quarter 
                check = [q[-1]]
                try:
                    # Try to search for all the monthly returns that fall into that quarter 
                    # If found, take cumulative products of this subset, taking the last entry minus one 
                    # as the quarterly return for that asset
                    q_ret = (returns[name][[returns[name].loc[i, "Month"] in q for 
                                                i in range(len(returns[name]))]].MonthlyReturn.add(1).cumprod(axis=0).iloc[-1])-1

                    # Update check array with the quarterly return. Convert check to pandas series
                    check.append(q_ret)
                    check = pd.Series(check)
                except (IndexError):
                    # If no stocks are found for that quarter, continue for-loop to next quarter 
                    continue
                # Add row of information to dataframe b
                b = b.append(check, ignore_index = True)    
        # Once returns calculated for all available quarters, assign q_returns dictionary that information under the asset name key
        q_returns[name] = b
        q_returns[name].columns = ["Date", "QuarterReturn"]
    # Return q_returns once run across all available assets 
    return q_returns

In [228]:
prices_dict = drop_dups(prices_dict, prices_dict.keys())

In [ ]:
stable_stock_dict = get_stable_stocks(prices_dict, prices_dict.keys())

In [266]:
prices_dict_usd = price_to_usd(stable_stock_dict, stable_stock_dict.keys())

In [269]:
# Calculate the gross USD price of each stock in prices_dict 
for stock in prices_dict_usd.keys():
    prices_dict_usd[stock]["price/adj*trfd"] = (prices_dict_usd[stock]["PriceUSD"]/
                                                prices_dict[stock]["ajexdi"])*prices_dict[stock]["trfd"]
    
prices_dict_usd

{'PLDT INC':            PriceUSD price/adj*trfd
 datadate                          
 20050103  24.744104      26.696098
 20050104   24.75953      26.712742
 20050105  24.730895      26.681847
 20050106  24.088889      25.989195
 20050107  23.943036      25.831836
 ...             ...            ...
 20211227  37.022292     115.506178
 20211228   36.38878      113.52968
 20211229  36.645111     114.329411
 20211230  36.666667     114.396661
 20211231  35.536378     110.870264
 
 [4368 rows x 2 columns],
 'ANGLO AMERICAN PLATINUM LTD':             PriceUSD price/adj*trfd
 datadate                           
 20050103   37.472382       72.33519
 20050104   36.380717      70.227883
 20050105   36.962025      71.350018
 20050106   37.992221      73.338667
 20050107   37.492821      72.374646
 ...              ...            ...
 20211227  114.656427      344.93472
 20211228  114.442667     344.291642
 20211229  113.748988     342.204763
 20211230  114.658334      344.94046
 20211231  113.60

In [272]:
daily_rets = daily_returns(prices_dict_usd, prices_dict_usd.keys())

In [275]:
monthly_rets = monthly_returns(daily_rets, daily_rets.keys())

In [278]:
quarterly_rets = quarterly_returns(monthly_rets, monthly_rets.keys())

In [280]:
# Function that retrieves fundamentals and the next six-month returns for a given time period
def get_date_info(df, date, date1, date2):
    # Create empty dataframe for row of information
    info = pd.DataFrame()
    for name in quarterly_rets.keys():
        # Retrieve fundamental date for that asset
        entry = df[name]
        try:   
            # Try searching for row of D/E, ROA, ROE, and SGI for that given date and convert to a list
            date_info = entry[entry.index == date].loc[:, ["D/E", "ROA", "ROE", "SGI"]].values.tolist()[0]
            # Create an array of the two next quarter returns of that asset
            array = [quarterly_rets[name][quarterly_rets[name].Date == date1].values.tolist()[0][-1],
                quarterly_rets[name][quarterly_rets[name].Date == date2].values.tolist()[0][-1]]
            # Increment array  by one 
            array = [x+1 for x in array]
            # Calculate 6-month return for that asset 
            ret = (np.cumprod(array)[-1]-1)
        except (RuntimeError, TypeError, NameError, IndexError):
            # If there are any exceptions, continue onto the next available asset
            continue
        
        # Add asset name and forward 6-month return to the fundamental list 
        date_info.append(name)
        date_info.append(ret)
        new_row = pd.Series(date_info)
        # If there are any pieces of missing data from the row, do not include that asset and continue onto next one
        if (new_row.isna().any().any() == True):
            continue
        else:
            # Otherwise, append info with this asset's information
            info = info.append(new_row, ignore_index = True)  
    # Set column names to relevant names and set the index to the asset names available 
    info.columns = ["D/E", "ROA", "ROE", "SGI", "Name", "Ret"]
    info = info.set_index("Name")
    # Return the full information for that given time period 
    return info

In [281]:
bbc = get_date_info(test_dict, 20070331, "200706", "200709")

# Function that ranks returns into quintiles and the other factors into terciles
def rank_factors(info):
    for item in info.columns.tolist():
        string = item + " Rank"
        if (item == "Ret"):
            info[string] = pd.qcut(info[item], 5, labels = [item + "1", item + "2", item + "3",
                                                         item + "4", item + "5"])
        else:
             info[string] = pd.qcut(info[item], 3, labels = [item + "1", item + "2", item + "3"])
    return info
sup = rank_factors(bbc)

In [344]:
# Functions that creates an array of lists of factors for each available asset
def fpg_prep(info):  
    # Drop the columns that do not contain the rankings 
    state = info.drop(["D/E", "ROA", "ROE", "SGI", "Ret"], axis=1)
    # Drop the returns ranked column and assign remaining info to new
    new = state.drop("Ret Rank", axis = 1)
    # Reset the index of state and drop the names column 
    state = state.reset_index().drop("Name", axis=1)

    final = []
    # For each row, append final with each row as an array of its own
    for i in range(len(state)):
        final.append(state.loc[i, state.columns].tolist())
    # Return both final and new 
    return final, new

groups, ranks = fpg_prep(sup)


,D/E Rank,ROA Rank,ROE Rank,SGI Rank
Name,,,,
PLDT INC,D/E3,ROA2,ROE3,SGI2
ANGLO AMERICAN PLATINUM LTD,D/E2,ROA3,ROE3,SGI3
SIBANYE-STILLWATER LIMITED,D/E2,ROA3,ROE3,SGI3
AFRICAN RAINBOW MINERALS LTD,D/E1,ROA3,ROE3,SGI3
KUALA LUMPUR KEPONG BHD,D/E2,ROA2,ROE1,SGI2
...,...,...,...,...
SCG PACKAGING PLC,D/E2,ROA1,ROE1,SGI2
REDE D OR SAO LUIZ SA,D/E3,ROA1,ROE3,SGI3
JD HEALTH INTERNATIONAL INC,D/E1,ROA1,ROE1,SGI2


In [283]:
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(groups).transform(groups)
eg = pd.DataFrame(te_ary, columns=te.columns_)
high_returns = eg.Ret5
eg   

,D/E1,D/E2,D/E3,ROA1,ROA2,ROA3,ROE1,ROE2,ROE3,Ret1,Ret2,Ret3,Ret4,Ret5,SGI1,SGI2,SGI3
0,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False
1,True,False,False,False,False,True,False,False,True,False,True,False,False,False,False,True,False
2,True,False,False,False,True,False,False,True,False,False,False,True,False,False,False,False,True
3,True,False,False,False,True,False,False,True,False,False,True,False,False,False,True,False,False
4,True,False,False,True,False,False,True,False,False,False,False,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,False,True,False,False,True,False,False,True,False,True,False,False,False,False,False,False,True
351,False,False,True,False,False,True,False,False,True,False,True,False,False,False,True,False,False
352,False,False,True,False,True,False,False,True,False,True,False,False,False,False,False,True,False
353,False,False,True,False,True,False,False,False,True,True,False,False,False,False,True,False,False


In [284]:
# Functions that mines assocation and lift rules, the 5th quintile returns and the true/false df for the FPG algorithm
def rules(final):    
    from mlxtend.preprocessing import TransactionEncoder
    # Preprocessing of input argument into true and false for each discretisation 
    te = TransactionEncoder()
    te_ary = te.fit(final).transform(final)
    # Eg is the true/false dataframe in this form for the fp growth algorithm
    eg = pd.DataFrame(te_ary, columns=te.columns_)
    # True/false column of the highest quintile of returns 
    high_returns = eg.Ret5

    # Finding frequent items in data for a minimum support of 5%
    from mlxtend.frequent_patterns import fpgrowth
    freq_items = fpgrowth(eg, min_support=0.05, use_colnames = True)

    from mlxtend.frequent_patterns import association_rules
    # Discover association and causal rules 
    asso_rules = association_rules(freq_items, metric="confidence", min_threshold=0.2)
    lift_rules = association_rules(freq_items, metric="lift", min_threshold=1.2)

    # Return true\false dataframe, high returns, and the found rules
    return eg, high_returns, asso_rules, lift_rules

eg, high_returns, asso_rules, lift_rules = rules(groups)
lift_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(D/E3),(ROA1),0.332394,0.335211,0.166197,0.500000,1.491597,0.054775,1.329577
1,(ROA1),(D/E3),0.335211,0.332394,0.166197,0.495798,1.491597,0.054775,1.324085
2,(ROE1),(D/E3),0.335211,0.332394,0.123944,0.369748,1.112377,0.012521,1.059268
3,(D/E3),(ROE1),0.332394,0.335211,0.123944,0.372881,1.112377,0.012521,1.060069
4,"(SGI1, D/E3)",(ROA1),0.112676,0.335211,0.084507,0.750000,2.237395,0.046737,2.659155
...,...,...,...,...,...,...,...,...,...
329,(ROA1),"(Ret5, SGI1)",0.335211,0.078873,0.050704,0.151261,1.917767,0.024265,1.085288
330,"(Ret5, ROE1)",(ROA1),0.067606,0.335211,0.059155,0.875000,2.610294,0.036493,5.318310
331,"(Ret5, ROA1)",(ROE1),0.087324,0.335211,0.059155,0.677419,2.020873,0.029883,2.060845
332,(ROE1),"(Ret5, ROA1)",0.335211,0.087324,0.059155,0.176471,2.020873,0.029883,1.108249


In [349]:
# Function that keeps the rules that imply Return 5 (highest 20% of stocks)
def rules_trim(asso_rules):    
    # For each row, if row does not contain "Ret5", then drop that row from the asso_rules list
    for index in asso_rules.index.tolist():
        if (list(asso_rules.loc[index, 'consequents'])[0] not in ["Ret5"]):
            asso_rules = asso_rules.drop(index, axis=0)
    return asso_rules

hello = rules_trim(lift_rules).sort_values("lift", ascending = False)
hello

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
152,(ROE1),"(Ret5, ROA1)",0.334129,0.073986,0.062053,0.185714,2.510138,0.037332,1.137211
153,(ROA1),"(Ret5, ROE1)",0.334129,0.073986,0.062053,0.185714,2.510138,0.037332,1.137211
146,(ROE3),"(Ret5, ROA3)",0.332936,0.065632,0.051313,0.154122,2.348257,0.029461,1.104612
147,(ROA3),"(Ret5, ROE3)",0.332936,0.065632,0.051313,0.154122,2.348257,0.029461,1.104612
139,(SGI3),(Ret5),0.332936,0.200477,0.076372,0.229391,1.144223,0.009626,1.037520
150,"(ROE1, ROA1)",(Ret5),0.278043,0.200477,0.062053,0.223176,1.113223,0.006311,1.029220
141,(ROE1),(Ret5),0.334129,0.200477,0.073986,0.221429,1.104507,0.007000,1.026910
143,(ROA1),(Ret5),0.334129,0.200477,0.073986,0.221429,1.104507,0.007000,1.026910


In [286]:
def high_ret_rules(asso_rules):    
    factors = []
    # For each row, if row contains "Ret5", then add that row's antecedents to the factors array
    for index in asso_rules.index.tolist():
        if (list(asso_rules.loc[index, 'consequents'])[0] in ["Ret5"]):
            factors.append(list(asso_rules.loc[index, 'antecedents']))
    return factors

In [350]:
asso_factors = high_ret_rules(asso_rules)
lift_factors = high_ret_rules(rules_trim(lift_rules).sort_values("lift", ascending = False))

# Make sure that generated association and lift rules 
unique_asso = [i for i in np.unique(asso_factors)]
unique_lift = [i for i in np.unique(lift_factors)]

def get_rules(unique_asso):
    associatons = []
    for asso in unique_asso:
        # Check if each antecedent is in the right format (list)
        if (isinstance(asso, list) == False):
            associatons.append([asso])
        else:
            associatons.append(asso)
    # Return list of lists
    return associatons

/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


[['ROA1'], ['ROA3'], ['ROE1'], ['ROE1', 'ROA1'], ['ROE3'], ['SGI3']]

In [289]:
# Function that retrieves the fair data set for given rules
def get_fair_datasets(true_control, false_control): 
    # Initialise true matches and false matches array
    true_match = []
    false_match = []
    # For every row in the control set that has the rules being true, check if the control variables 
    # from the false set are the same. If so, add row index to true_match and false_match 
    for i in true_control.index:
        for j in false_control.index:
            if (true_control.loc[i, :].tolist() == false_control.loc[j, :].tolist()):
                true_match.append(i)
                false_match.append(j)
    # Return the dates of the fair datasets
    return true_match, false_match

In [ ]:
# Function to get the odds ratio confidence interval for the proposed rule at a 90% confidence level 
def get_oddsratio_CI(exposure, non_exposure, returns):
    # Count for number of times both exposure and non-exposure groups have the consequent 
    n11 = 0
    # Count for number of times the exposure has the consequent and non-exposure groups does not  
    n12 = 0
    # Count for number of times the exposure does not have the consequent and non-exposure groups does
    n21 = 0
    # Count for number of times both exposure and non-exposure groups do not have the consequent
    n22 = 0
    
    for i in range(len(exposure)):
        # If both the exposure and non exposure groups have returns in the 5th quantile, increment n11 by one
        if (returns.loc[exposure.index[i]] == True) and (returns.loc[non_exposure.index[i]] == True):
            n11 += 1
        elif (returns.loc[exposure.index[i]] == True) and (returns.loc[non_exposure.index[i]] == False):
            n12 += 1
        elif (returns.loc[exposure.index[i]] == False) and (returns.loc[non_exposure.index[i]] == True):
            n21 += 1
        elif (returns.loc[exposure.index[i]] == False) and (returns.loc[non_exposure.index[i]] == False):
            n22 += 1

    # To ensure that you are not dividing by 0, if n12 or n21 are zero, set them to one
    if n21 == 0:
        n21 = 1
    if n12 == 0:
        n12 = 1

    # Calculate the odds ratio point estimate 
    odds_ratio = n12/n21

    # Compute the lower and upper bounds of the odds ratio's conficence interval 
    lower_bound = np.exp(np.log(odds_ratio) - (1.64*np.sqrt((1/n12) + (1/n21))))
    upper_bound = np.exp(np.log(odds_ratio) + (1.64*np.sqrt((1/n12) + (1/n21))))
    
    #
    return lower_bound, upper_bound

In [352]:
# Function that mines causal rules from established associations 
def get_causal_rules(eg, rules, returns):
    # Array to store mined causal rules 
    causal_rules = []
    for rule in rules:
        # If all four factors are included in the associations, include in the causal rules
        if (len(rule) == 4):
            causal_rule.append(rule)
            continue
        else:
            # Otherwise, for each association antecedent, search rows for when antecedent and true and false
            trues = list(np.ones(len(rule), dtype=bool))
            falses = list(np.zeros(len(rule), dtype=bool))
            true = eg[[eg[rule].iloc[i].tolist() == trues for i in range(len(eg[rule]))]]
            false = eg[[eg[rule].iloc[i].tolist() == falses for i in range(len(eg[rule]))]]
            
            # Remove the returns columns and columns with antecedants in question. Only the control variables remain
            remove = ["Ret1", "Ret2", "Ret3", "Ret4", "Ret5"]
            for cond in rule:
                remove.append(cond[:3] + "1")
                remove.append(cond[:3] + "2")
                remove.append(cond[:3] + "3")
            true_control = true.drop(remove, axis = 1)
            false_control = false.drop(remove, axis = 1)
            
            # Drop duplicates from true and false control sets to ensure that there is at most one set of matching rows
            true_control = true_control.drop_duplicates(subset = true_control.columns, keep='first')
            false_control = false_control.drop_duplicates(subset = false_control.columns, keep='first')
            
            # Retrieve the date indices from the fair datasets 
            true_match, false_match = get_fair_datasets(true_control, false_control)    
            
            # Getting returns columns for the rows of the fair dataset 
            exposure_returns = returns[[i in true_match for i in range(len(returns))]]
            non_exposure_returns = returns[[i in false_match for i in range(len(returns))]]
            
            # Compute the bounds of the rule's odd ratio confidence interval
            lower, upper = get_oddsratio_CI(exposure_returns, non_exposure_returns, returns)
            
            #print(rule)
            #print("Odds Ratio Lower Bound: ", lower)
            #print("Odds Ratio Upper Bound: ", upper)
            
            if (lower > 0.5):
                causal_rules.append(rule)
    
    return causal_rules

get_causal_rules(eg, unique_asso, high_returns)   

[['D/E3'], ['ROE1'], ['SGI3']]

In [296]:
next_period_returns = get_date_info(test_dict, 20110930, "201112", "201203")
ranked = rank_factors(next_period_returns)

___ , factors = fpg_prep(ranked)
factors

,D/E Rank,ROA Rank,ROE Rank,SGI Rank
Name,,,,
PLDT INC,D/E3,ROA3,ROE3,SGI1
ANGLO AMERICAN PLATINUM LTD,D/E1,ROA1,ROE1,SGI2
SIBANYE-STILLWATER LIMITED,D/E1,ROA3,ROE1,SGI3
AFRICAN RAINBOW MINERALS LTD,D/E1,ROA3,ROE2,SGI2
KUALA LUMPUR KEPONG BHD,D/E1,ROA3,ROE3,SGI2
...,...,...,...,...
ZHEJIANG CENTURY HUATONG GRP,D/E1,ROA2,ROE1,SGI1
SUN ART RETAIL GROUP LTD,D/E3,ROA1,ROE2,SGI3
PARADE TECHNOLOGIES LTD,D/E1,ROA3,ROE2,SGI2


In [353]:
# Function that finds all stocks that contain the found rules
def get_stocks(factors, rules):
    # Array to store names of companies that appease rules
    checker = []
    for stock in factors.index.unique():
        for factor in rules:
            # If individual rule is a suset of the whole row of factors, then that 
            # company contains the observed rule and is added the checker array
            if (set(factor).issubset(set(factors.loc[stock].values.tolist())) == True) :
                checker.append(stock)
    # Convert checker array to dataframe - then only keep the unique names found
    checker = pd.DataFrame(checker, columns = ["Name"])
    checker = checker.Name.unique()
    # Return unique names found
    return checker

names = get_stocks(factors, unique_lift)

array(['PLDT INC', 'ANGLO AMERICAN PLATINUM LTD',
       'SIBANYE-STILLWATER LIMITED', 'AFRICAN RAINBOW MINERALS LTD',
       'PPB GROUP BHD', 'JD.COM INC', 'ALIBABA GROUP HLDG',
       'ENEL CHILE SA', 'ZTO EXPRESS CAYMAN INC', 'ENEL AMERICAS SA',
       'GRUPO TELEVISA SAB', 'MANILA ELECTRIC CO',
       'KEPCO-KOREA ELEC POWER CORP', 'FOMENTO ECONOMICO MEXICANO',
       'BERGER PAINTS INDIA LTD', 'MARICO LTD', 'GENTING BHD',
       'SASOL LTD', 'HAP SENG CONSOLIDATED BHD', 'SIME DARBY BHD',
       'MISC BERHAD', 'TIGER BRANDS LTD', 'NESTLE (MALAYSIA) BHD',
       'GENTING MALAYSIA BHD', 'IOI CORP BHD',
       'CHAROEN POKPHAND FOODS PCL', 'INTOUCH HOLDINGS PCL',
       'SIAM CEMENT PCL', 'ADVANCED INFO SERVICE PCL',
       'SAMSUNG SDI CO LTD', 'HYUNDAI ENGR & CONSTR CO',
       'GS ENGINEERING & CONSTRUCTN', 'HYUNDAI STEEL CO',
       'SAMSUNG ELECTRO-MECHANICS CO', 'LG CORP',
       'KOREAN AIR LINES CO LTD', 'HANWHA SOLUTIONS CORP',
       'BTS GROUP HOLDINGS PCL', 'TENAGA NASIONA

In [ ]:
def six_month_returns(rets, stock_names, date1, date2):
    returns = []
    available_stocks = len(stock_names)
    for stock in stock_names:
        array = [rets[stock][rets[stock].Date == date1].values.tolist()[0][-1],
                        rets[stock][rets[stock].Date == date2].values.tolist()[0][-1]]
        array = [x+1 for x in array]
        returns.append((np.cumprod(array)[-1]-1))

    if (available_stocks == 0):
        return 0.0
    #print("Length of Returns: ", len(returns))
    #print("Number of Stocks: ", len(stock_names))

    return (sum(returns)/available_stocks)

In [197]:
# Function to calculate the six-month equal-weighted portfolio returns 
def six_month_returns1(rets, stock_names, date1, date2):
    # Array to store the six-month returns of each stock in the portfolio
    returns = []
    # Counter to number of stocks available 
    available_stocks = len(stock_names)
    for stock in stock_names:
        # For each stock, check if the six_month return information is available. If so, calculate six-month returns 
        # and append the returns array
        try:
            array = [rets[stock][rets[stock].Date == date1].values.tolist()[0][-1],
                        rets[stock][rets[stock].Date == date2].values.tolist()[0][-1]]
            array = [x+1 for x in array]
            returns.append((np.cumprod(array)[-1]-1))
        # If stock is not available, reduce the number of stocks in the portfolio and continue to the next stock
        except IndexError:
            available_stocks -= 1
            continue

    #print("Length of Returns: ", len(returns))
    #print("Number of Stocks: ", len(stock_names))
    
    # Returnb the portfolio 6-month return
    return (sum(returns)/available_stocks)

In [355]:
# Setting up years for the simulation period
years = np.arange(2005, 2022)
dates = []
# For each year, create the ending dates of the four quarters 
for year in years:
    q1 = str(year) + "0331"
    q2 = str(year) + "0630"
    q3 = str(year) + "0930"
    q4 = str(year) + "1231"
    # Add dates to dates array
    dates.append(int(q1))
    dates.append(int(q2))
    dates.append(int(q3))
    dates.append(int(q4))


In [335]:
# Simulation start at Q1 2006 (position four in the dates array above)
i = 4
# Array to store quarterly dates and returns of the rule mining strategies 
records4 = []
records5 = []
records6 = []

# Store the names of all portfolio formation throghout the simulation's history
all_asso_names = []
all_lift_names = []
all_causal_names = []
# While there are still two dates ahead of the rebalancing date; mine rules, rebalance portfolios, and calculate returns
while (i < 66):
    # Retrieve fundamental information from six months ago and their 6-month forward return
    info = get_date_info(test_dict, dates[i-2], str(dates[i-1])[:6], str(dates[i])[:6])

    # Retrieve the ranked information
    groups, ranks = fpg_prep(rank_factors(info))
    # Retrieve the true/false dataframe, high returns column, and association and lift rules for these ranks factors 
    eg, high_returns, asso_rules, lift_rules = rules(groups)
    
    # Keep only the rules that imply Return 5
    asso_factors = high_ret_rules(asso_rules)
    lift_factors = high_ret_rules(rules_trim(lift_rules).sort_values("lift", ascending = False))

    # Keep only the unique antecedents and make sure it is in the format list of lists
    unique_asso = [i for i in np.unique(asso_factors)]
    unique_lift = [i for i in np.unique(lift_factors)]
    unique_lift = get_rules(unique_lift)
    unique_asso = get_rules(unique_asso)
    
    # From the unique association antecedents, mine causal rules 
    causal_rules = get_causal_rules(eg, unique_asso, high_returns)

    #print(unique_asso)
    #print(unique_lift)
    
    # Get the fundamental information at the rebalance (current) date and discretise
    next_period_returns = get_date_info(test_dict, dates[i], str(dates[i+1])[:6], str(dates[i+2])[:6])
    ranked = rank_factors(next_period_returns)
    ___ , factors = fpg_prep(ranked)
    
    # Get names of stocks that satisfy the lift rules 
    lift_names = get_stocks(factors, unique_lift)
    
    # If the names or rules are non-zero, rebalance portfolios and calculate returns 
    if (len(lift_names) != 0 or len(unique_lift) != 0):
        rebalance = "Yes"
        all_lift_names.append(lift_names)
        add1 = [str(dates[i+2])[:6], len(unique_lift), len(lift_names), six_month_returns1(quarterly_rets, lift_names, 
                                                  str(dates[i+1])[:6], str(dates[i+2])[:6]), rebalance]
    # If no names or rules found, then do not rebalance the portfolio and calculate the six month 
    # returns of the same portfolio as the previous period
    elif (len(lift_names) == 0 or len(unique_lift) == 0):
        rebalance = "No"
        add1 = [str(dates[i+2])[:6], len(causal_rules), len(all_lift_names[-1]), six_month_returns1(quarterly_rets, 
                        all_lift_names[-1],  str(dates[i+1])[:6], str(dates[i+2])[:6]), rebalance]
    
    asso_names = get_stocks(factors, unique_asso)
    # If the names or rules are non-zero, rebalance portfolios and calculate returns 
    if (len(asso_names) != 0 or len(unique_asso) != 0):
        rebalance = "Yes"
        all_asso_names.append(asso_names)
        add2 = [str(dates[i+2])[:6], len(unique_asso), len(asso_names), six_month_returns1(quarterly_rets, asso_names, 
                                                str(dates[i+1])[:6], str(dates[i+2])[:6]), rebalance]
     # If no names or rules found, then do not rebalance the portfolio and calculate the six month 
    # returns of the same portfolio as the previous period
    elif (len(asso_names) == 0 or len(unique_asso) == 0):
        rebalance = "No"
        add2 = [str(dates[i+2])[:6], len(causal_rules), len(all_asso_names[-1]), six_month_returns1(quarterly_rets, 
                        all_asso_names[-1],  str(dates[i+1])[:6], str(dates[i+2])[:6]), rebalance]
    
    causal_names = get_stocks(factors, causal_rules)
    # If the names or rules are non-zero, rebalance portfolios and calculate returns 
    if (len(causal_names) != 0 or len(causal_rules) != 0):
        rebalance = "Yes"
        all_causal_names.append(causal_names)
        add3 = [str(dates[i+2])[:6], len(causal_rules), len(causal_names), six_month_returns1(quarterly_rets, 
                          causal_names, str(dates[i+1])[:6], str(dates[i+2])[:6]), rebalance]
    # If no names or rules found, then do not rebalance the portfolio and calculate the six month 
    # returns of the same portfolio as the previous period
    elif (len(causal_names) == 0 or len(causal_rules) == 0):
        rebalance = "No"
        add3 = [str(dates[i+2])[:6], len(causal_rules), len(all_causal_names[-1]), six_month_returns1(quarterly_rets, 
                        all_causal_names[-1],  str(dates[i+1])[:6], str(dates[i+2])[:6]), rebalance]
 
    # Append arrays with date, number of rules found, number of stocks in portfolio, 6M returns, and if there was rebalancing
    records4.append(add1)
    records5.append(add2)
    records6.append(add3)
    i+=2


/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarray

/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarray

In [357]:
# For each strategy's returns, create dataframe of results
stuff4 = pd.DataFrame(records4, columns = ["Date", "Number of Rules", "Number of Stocks", "6M Returns", "Rebalance?"])
stuff4 = stuff4.set_index("Date")
stuff4

stuff5 = pd.DataFrame(records5, columns = ["Date", "Number of Rules", "Number of Stocks", "6M Returns", "Rebalance?"])
stuff5 = stuff5.set_index("Date")
stuff5

stuff6 = pd.DataFrame(records6, columns = ["Date", "Number of Rules", "Number of Stocks", "6M Returns", "Rebalance?"])
stuff6 = stuff6.set_index("Date")
stuff4

,Number of Rules,Number of Stocks,6M Returns,Rebalance?
Date,,,,
200609,6,217,0.275187,Yes
200703,9,279,0.651629,Yes
200709,8,235,0.692008,Yes
200803,6,300,-0.076007,Yes
200809,4,247,-0.254247,Yes
200903,8,336,0.038374,Yes
200909,7,321,0.525510,Yes
201003,10,410,0.308702,Yes
201009,7,436,0.223522,Yes


In [338]:
# Printing cumulative returns for each strategy
print(stuff6["6M Returns"].mean())
print((stuff4["6M Returns"].add(1).cumprod(axis = 0)[-1])-1)
print((stuff5["6M Returns"].add(1).cumprod(axis = 0)[-1])-1)
print((stuff6["6M Returns"].add(1).cumprod(axis = 0)[-1])-1)

0.17125369191055126
66.41780819920496
53.826947923896796
75.55780245497127


In [339]:
# Calculating and printing annualised returns for each strategy
annualised_return_4 = ((stuff4["6M Returns"].add(1).cumprod(axis = 0)[-1])**(1/16) - 1)*100
print("Lift rules returns: ", round(annualised_return_4, 2))

annualised_return_5 = ((stuff5["6M Returns"].add(1).cumprod(axis = 0)[-1])**(1/16) - 1)*100
print("Association rules returns: ", round(annualised_return_5, 2))

annualised_return_6 = ((stuff6["6M Returns"].add(1).cumprod(axis = 0)[-1])**(1/16) - 1)*100
print("Causal AR returns: ", round(annualised_return_6, 2))


Lift rules returns:  30.11
Association rules returns:  28.44
Causal AR returns:  31.14


In [ ]:
from sklearn.linear_model import LinearRegression

x = np.array(stuff5["6M Returns"]).reshape((-1,1))
x

y = np.array(stuff6["6M Returns"])
y

model = LinearRegression().fit(x, y)
model.intercept_

In [ ]:
print(stuff4["6M Returns"].describe())
1.167445**2

In [ ]:
def sim():  
    import random
    i = 4
    records4 = []
    records5 = []
    records6 = []

    all_asso_names = []
    all_lift_names = []
    all_causal_names = []
    while (i < 66):
        info = get_date_info(test_dict, dates[i-2], str(dates[i-1])[:6], str(dates[i])[:6])

        sup = rank_factors(info)
        groups, ranks = fpg_prep(sup)
        eg, high_returns, asso_rules, lift_rules = rules(groups)

        asso_factors = high_ret_rules(asso_rules)
        lift_factors = high_ret_rules(rules_trim(lift_rules).sort_values("lift", ascending = False))

        unique_asso = [i for i in np.unique(asso_factors)]
        unique_lift = [i for i in np.unique(lift_factors)]

        unique_lift = get_rules(unique_lift)
        unique_asso = get_rules(unique_asso)

        causal_rules = get_causal_rules(eg, unique_asso, high_returns)

            #print(unique_asso)
            #print(unique_lift)

        next_period_returns = get_date_info(test_dict, dates[i], str(dates[i+1])[:6], str(dates[i+2])[:6])
        ranked = rank_factors(next_period_returns)

        ___ , factors = fpg_prep(ranked)

        lift_names = get_stocks(factors, unique_lift)

        if (len(lift_names) != 0 or len(unique_lift) != 0):
            rebalance = "Yes"
            all_lift_names.append(lift_names)
            add1 = [str(dates[i+2])[:6], len(unique_lift), len(lift_names), six_month_returns1(quarterly_rets, 
                                list(random.sample(set(lift_names), min(250, len(lift_names)))), str(dates[i+1])[:6], str(dates[i+2])[:6]), rebalance]
        elif (len(lift_names) == 0 or len(unique_lift) == 0):
            rebalance = "No"
            #previous_names = list(set(factors.index.unique()) & set(all_lift_names[-1]))
            add1 = [str(dates[i+2])[:6], len(causal_rules), len(all_lift_names[-1]), six_month_returns1(quarterly_rets, 
                           list(random.sample(set(all_lift_names[-1]), min(250, len(all_lift_names[-1])))),  str(dates[i+1])[:6], str(dates[i+2])[:6]), rebalance]

        asso_names = get_stocks(factors, unique_asso)

        if (len(asso_names) != 0 or len(unique_asso) != 0):
            rebalance = "Yes"
            all_asso_names.append(asso_names)
            add2 = [str(dates[i+2])[:6], len(unique_asso), len(asso_names), six_month_returns1(quarterly_rets, 
                            list(random.sample(set(asso_names), min(250, len(asso_names)))), str(dates[i+1])[:6], str(dates[i+2])[:6]), rebalance]
        elif (len(asso_names) == 0 or len(unique_asso) == 0):
            rebalance = "No"
            #previous_names = list(set(factors.index.unique()) & set(all_asso_names[-1]))
            add2 = [str(dates[i+2])[:6], len(causal_rules), len(all_asso_names[-1]), six_month_returns1(quarterly_rets, 
                            list(random.sample(set(all_asso_names[-1]), min(250,len(all_asso_names[-1])))), str(dates[i+1])[:6], str(dates[i+2])[:6]), rebalance]

        causal_names = get_stocks(factors, causal_rules)

        if (len(causal_names) != 0 or len(causal_rules) != 0):
            rebalance = "Yes"
            all_causal_names.append(causal_names)
            add3 = [str(dates[i+2])[:6], len(causal_rules), len(causal_names), six_month_returns1(quarterly_rets, 
                                  list(random.sample(set(causal_names), min(250, len(causal_names)))), str(dates[i+1])[:6], str(dates[i+2])[:6]), rebalance]
        elif (len(causal_names) == 0 or len(causal_rules) == 0):
            rebalance = "No"
            #previous_names = list(set(factors.index.unique()) & set(all_causal_names[-1]))
            add3 = [str(dates[i+2])[:6], len(causal_rules), len(all_causal_names[-1]), six_month_returns1(quarterly_rets, 
                                list(random.sample(set(all_causal_names[-1]), min(250, len(all_causal_names[-1])))), str(dates[i+1])[:6], str(dates[i+2])[:6]), rebalance]

        records4.append(add1)
        records5.append(add2)
        records6.append(add3)
        i+=2

    return records4, records5, records6

lift_ret, asso_ret, causal_ret = sim()

lift = pd.DataFrame(lift_ret, columns = ["Date", "Number of Rules", "Number of Stocks", "6M Returns", "Rebalance?"])
lift = lift.set_index("Date")


asso = pd.DataFrame(asso_ret, columns = ["Date", "Number of Rules", "Number of Stocks", "6M Returns", "Rebalance?"])
asso = asso.set_index("Date")

causal = pd.DataFrame(causal_ret, columns = ["Date", "Number of Rules", "Number of Stocks", "6M Returns", "Rebalance?"])
causal = causal.set_index("Date")
causal